In [1]:
#imports 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
#Sklearn imports 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, LabelBinarizer, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer , KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [3]:
#Load dataset
df_vpn1 = pd.read_json('vpn1.json')
df_vpn2 = pd.read_json('vpn2.json')
df_vpn3 = pd.read_json('vpn3.json')

In [4]:
def extracting_data(df,target):
    l_id = []
    l_bits = []
    l_direction = []
    l_packets_number = []
    l_timestamp = []
    l_target = []
    
    
    for i in range(len(df['x_packets'])):
        packets = df['x_packets'][i]
        for packet in packets:
            l_bits.append(packet['bytes'])
            l_direction.append(packet['direction'])
            l_packets_number.append(packet['packets_number'])
            l_timestamp.append(packet['timestamp'])
            l_id.append(df['id'][i])
            l_target.append(target)
            
    return l_id,l_bits,l_direction,l_packets_number,l_timestamp,l_target
    

In [5]:
#formated vp1 dataframe
l_id,l_bits,l_direction,l_packets_number,l_timestamp,l_target = extracting_data(df_vpn1,'vpn1')
formated_df_vp1 = pd.DataFrame(list(zip(l_id,l_bits,l_direction,l_packets_number,l_timestamp,l_target)), columns =['id', 'bytes','direction','packets_number','timestamp','target'])

#formated vpn2 dataframe
l_id,l_bits,l_direction,l_packets_number,l_timestamp,l_target = extracting_data(df_vpn2,'vpn2')
formated_df_vp2 = pd.DataFrame(list(zip(l_id,l_bits,l_direction,l_packets_number,l_timestamp,l_target)), columns =['id', 'bytes','direction','packets_number','timestamp','target'])

#formated vpn3 dataframe
l_id,l_bits,l_direction,l_packets_number,l_timestamp,l_target = extracting_data(df_vpn3,'vpn3')
formated_df_vp3 = pd.DataFrame(list(zip(l_id,l_bits,l_direction,l_packets_number,l_timestamp,l_target)), columns =['id', 'bytes','direction','packets_number','timestamp','target'])

1084.2026141360163

In [57]:
#Verify if extracting data is good 
print(len(l_id))
ll = []
for i in range(len(df_vpn3)):
    ll.append(len(df_vpn3['x_packets'][i]))
print(sum(ll))
    

1088387
1088387


In [6]:
df = pd.concat([formated_df_vp1,formated_df_vp2,formated_df_vp3]).reset_index()
df.drop('index',axis=1, inplace=True)
df.head()

,id,bytes,direction,packets_number,timestamp,target
0,DdpYMBdSWcwU7kN7NWjWPQ,183,1,1,0.000000,vpn1
1,DdpYMBdSWcwU7kN7NWjWPQ,52,-1,1,0.047616,vpn1
2,DdpYMBdSWcwU7kN7NWjWPQ,679,1,1,0.048298,vpn1
3,DdpYMBdSWcwU7kN7NWjWPQ,52,-1,1,0.048652,vpn1
4,DdpYMBdSWcwU7kN7NWjWPQ,167,-1,1,0.052448,vpn1


In [7]:
df['target'].value_counts()

vpn2    1364028
vpn3    1088387
vpn1     883195
Name: target, dtype: int64

In [69]:
df.columns

Index(['id', 'bytes', 'direction', 'packets_number', 'timestamp'], dtype='object')

In [8]:
#Preprocessing features
numerical_features = ['bytes', 'direction', 'packets_number', 'timestamp']
categorical_feautures = ['id']

numerical_pipline = make_pipeline(StandardScaler())
categorical_pipline = make_pipeline(OrdinalEncoder(unknown_value=-1,handle_unknown='use_encoded_value'),StandardScaler())

preprocessing = make_column_transformer((numerical_pipline,numerical_features),(categorical_pipline,categorical_feautures))

In [9]:
#Split data feautures and target 
features = ['id','bytes', 'direction', 'packets_number', 'timestamp']
target = ['target']

X = df[features]
y = df[target]

In [10]:
X_transformed = preprocessing.fit_transform(X)


In [11]:
X_transformed

array([[-0.95870354,  1.29363218, -0.27104436, -0.52762616, -0.82185015],
       [-1.09964138, -0.77301726, -0.27104436, -0.52518539, -0.82185015],
       [-0.42507632,  1.29363218, -0.27104436, -0.52515043, -0.82185015],
       ...,
       [-0.53589003,  1.29363218, -0.27104436, -0.52762483,  0.08374378],
       [-0.53589003,  1.29363218, -0.27104436, -0.52762365,  0.08374378],
       [-0.53589003,  1.29363218, -0.27104436, -0.52762211,  0.08374378]])

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.33, random_state=42)

In [155]:
rfc=DecisionTreeClassifier(random_state=42)

param_grid = { 
    'splitter': ['best','random'],
    'min_samples_split': [2,4,6],
    'max_depth' : [None,2,4],
    'criterion' :['gini', 'entropy','log_loss']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3)
CV_rfc.fit(X_train, y_train.values.ravel())

C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
54 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  warnings.warn(some_fits_failed_m

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 2, 4],
                         'min_samples_split': [2, 4, 6],
                         'splitter': ['best', 'random']})

In [ ]:
X_transformed = pd.DataFrame(X_transformed)

In [156]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'min_samples_split': 2,
 'splitter': 'best'}

In [13]:
from sklearn.tree import DecisionTreeClassifier
Fmodel = DecisionTreeClassifier(random_state=0,criterion='entropy')


In [14]:
Fmodel

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [15]:
Fmodel.fit(X_train,y_train.values.ravel())

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [16]:
#Scoring
from sklearn.metrics import f1_score
print(f'the accuracy of the model on the training set is {Fmodel.score(X_train, y_train)}')
print(f'the accuracy of the model on the validation set is {Fmodel.score(X_test, y_test)}')

the accuracy of the model on the training set is 1.0
the accuracy of the model on the validation set is 0.9984655944299897


In [17]:
Fmodel.fit(X_transformed,y.values.ravel())

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [18]:
df_test = pd.read_json('test_data.json')

In [19]:
l_id,l_bits,l_direction,l_packets_number,l_timestamp,l_target = extracting_data(df_test,None)
formated_df_test = pd.DataFrame(list(zip(l_id,l_bits,l_direction,l_packets_number,l_timestamp,l_target)), columns =['id', 'bytes','direction','packets_number','timestamp','target'])

In [20]:
formated_df_test.drop('target', axis=1, inplace=True)

In [21]:
X_test_transformed = preprocessing.transform(formated_df_test)
y_test_predict = Fmodel.predict(X_test_transformed)

In [22]:
y_test_predict

array(['vpn3', 'vpn3', 'vpn3', ..., 'vpn3', 'vpn3', 'vpn3'], dtype=object)

In [23]:
formated_df_test['label'] = y_test_predict

In [24]:
pre_out = formated_df_test.drop(['bytes','direction','packets_number','timestamp'],axis=1)

In [25]:
ids = pre_out['id'].unique()

In [26]:
labels = []
for i in range(len(ids)):
    nw_pd = pre_out[pre_out['id']== ids[i]]
    frequency = nw_pd['label'].value_counts()
    top_label = frequency.index[0]
    labels.append(top_label)


In [27]:
len(labels)

46

In [28]:
sub = pd.DataFrame(data={'id': ids, 'label': labels}).set_index("id")

In [29]:
sub.to_csv('submit2.csv')